[View in Colaboratory](https://colab.research.google.com/github/kimitakem/colab_ex/blob/master/sample.ipynb)

# Tensorflow Eager executiomモードのチュートリアル要約
## ライブラリのインポート

In [5]:
import tensorflow as tf
tf.enable_eager_execution()


## Tensorの演算
- 足し算：`tf.add`



In [9]:
print(tf.add(1,2))
print(tf.add([1,2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("Hello World"))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'SGVsbG8gV29ybGQ', shape=(), dtype=string)


ヘディングです